In [52]:
import torch
import torch.nn as nn
import tiktoken

In [54]:
from gpt_model import * 

In [46]:
tokenizer = tiktoken.get_encoding("gpt2")

In [47]:
model = GPTModel(GPT_CONFIG_124M)

In [48]:
start_context = "Every effort moves you"

In [51]:
token_ids = generate_text_sample(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

NameError: name 'generate_text_sample' is not defined

In [50]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [1107,  588, 11311]]) #  " really like chocolate"]

In [39]:
with torch.no_grad():
    logits = model(inputs)

probs = torch.softmax(logits, dim=-1) # Probability of each token in vocabulary
print(probs.shape) # Shape: (batch_size, num_tokens, vocab_size)

torch.Size([2, 3, 50257])


In [40]:
probs

tensor([[[3.3267e-05, 9.2453e-06, 2.3175e-05,  ..., 9.2133e-06,
          1.7504e-05, 1.5636e-05],
         [2.9523e-05, 1.4834e-05, 1.7589e-05,  ..., 1.4187e-05,
          1.3010e-05, 5.0947e-05],
         [3.3528e-05, 1.1574e-05, 2.7096e-05,  ..., 1.1489e-05,
          1.9096e-05, 1.5380e-05]],

        [[3.6627e-05, 3.0407e-05, 3.4317e-05,  ..., 9.9421e-06,
          1.0082e-05, 1.3281e-05],
         [1.3278e-05, 3.8525e-05, 1.9435e-05,  ..., 1.5135e-05,
          3.2835e-06, 1.7624e-05],
         [4.9017e-05, 9.7251e-06, 2.4899e-05,  ..., 9.7779e-06,
          4.7495e-05, 4.8987e-05]]])

In [41]:
# predicted tokens:
token_ids = torch.argmax(probs, dim=-1, keepdim=True)
print("Token IDs:\n", token_ids)

Token IDs:
 tensor([[[14702],
         [31366],
         [47759]],

        [[12321],
         [10998],
         [31722]]])


In [42]:
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

NameError: name 'token_ids_to_text' is not defined

In [56]:
print(f"Targets batch 2: {token_ids_to_text(targets[1], tokenizer)}")
print(f"Outputs batch 2: {token_ids_to_text(token_ids[1].flatten(), tokenizer)}")

NameError: name 'token_ids_to_text' is not defined

In [57]:
targets

tensor([[ 3626,  6100,   345],
        [ 1107,   588, 11311]])

In [58]:
text_idx = 0
target_probs_1 = probs[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 1:", target_probs_1)

text_idx = 1
target_probs_2 = probs[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 2:", target_probs_2)

Text 1: tensor([1.8344e-05, 8.5335e-06, 1.0164e-05])
Text 2: tensor([7.7210e-06, 1.0019e-05, 1.8577e-05])


In [59]:
torch.set_printoptions( sci_mode=False )

In [60]:
# Compute logarithm of all token probabilities
log_probs = torch.log(torch.cat((target_probs_1, target_probs_2)))
print(log_probs)

tensor([-10.9062, -11.6715, -11.4967, -11.7716, -11.5111, -10.8936])


In [61]:
avg_log_probs = torch.mean(log_probs)
print(avg_log_probs)
neg_avg_log_probs = avg_log_probs * -1
print(neg_avg_log_probs)


tensor(-11.3751)
tensor(11.3751)
